In [1]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, JSON
import requests
import time

# Basic configuration
endpoint = "https://wb.pcp-on-web.de/store-oker/research/"

# This function counts the genrated tripples of the research ontolgy (data:)
def count_triples(endpoint,graph) :
    #count number of exiting triples
    sparql = SPARQLWrapper(endpoint+'sparql')
    sparql.setQuery('SELECT (COUNT(*) as ?Triples) FROM '+graph+' WHERE { ?s ?p ?o}')
    sparql.setReturnFormat(JSON)
    json_results = sparql.query().convert()
    return json_results['results']['bindings'][0]['Triples']['value']
    

# This function executes a sparql construct query and imports the data to the research ontolgy (data:)
def query_sparql(query, endpoint) :
    start_time = time.time()
    
    #perform construct Query
    sparql = SPARQLWrapper(endpoint+'sparql')
  
    #query += " LIMIT 10"
    sparql.setQuery(query)
    sparql.setReturnFormat(RDFXML)
    sparql.setOnlyConneg(True)
    resultsets = sparql.query().convert()
    result = resultsets.serialize(format='ttl')
    
    # Add PREFIXs to data to insert
    data=""
    data_graph="leer"
    text_lines = query.split("\n")
    info = text_lines[1]+"\n"+text_lines[2]
    for line in text_lines:
        if line.find("PREFIX ")==0:
           data += line + "\n"
           p = 0
           p = line.find("PREFIX data:") 
           if p == 0 :
                data_graph = line[line.find("<"):]
            
    # Add SPARQL INSERT to data to insert
    data += "INSERT DATA {\nGRAPH data:\n{\n"
    text_lines = result.split("\n")
    for line in text_lines:
        if line.find("@prefix ")==-1:
           data += line + "\n"
    # Close
    data += "}}"
   
    start_count = count_triples(endpoint,data_graph)
    
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(data)
    sparql.method = 'POST'
    sparql.query()

    end_count = count_triples(endpoint,data_graph)

    output =  "\n"+info+"\n"+start_count + "-" + end_count + "\n--- " + str(time.time() - start_time) + " seconds ---"
    return output


In [2]:
data = """
# Basic Import of Lecturers including relevant attributes and pictures
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>

CONSTRUCT   {

?lecturer rdf:type pcp:Lecturer.
?lecturer pcp:derivedFrom ?professor.

?lecturer pcp:gnd ?gnd.
?lecturer rdfs:label ?label .
?lecturer pcp:note ?note .
?lecturer pcp:furtherInformation ?furtherInformation .
?lecturer pcp:additionToSurname ?additionToSurname .
?lecturer pcp:alternativeWritingOfSurname ?alternativeWritingOfSurname .
?lecturer pcp:forename ?forename .
?lecturer pcp:surname ?surname .
?lecturer rdfs:seeAlso ?webLink .
?lecturer rdfs:seeAlso ?seeAlso .


}


WHERE {
  
  GRAPH bamberg: {  

      # Import all Professors relevant for Early Modern History
      ?professor a cpm:Professor.
      ?professor cpm:hasPeriod ?prof_birth.
      ?prof_birth a cpm:Birth.
      ?prof_birth cpm:date ?prof_birth_date .
      BIND (xsd:integer(substr(str(?prof_birth_date), 1, 4)) as ?prof_birth_year)       
      FILTER (!BOUND(?prof_birth_year) || ((?prof_birth_year>1450) && (?prof_birth_year<1800)))
      BIND (IRI(REPLACE(REPLACE(str(?professor),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?lecturer). 

      #retreive label from professor
      OPTIONAL { ?professor cpm:surname ?surname.}
      OPTIONAL { ?professor cpm:forename ?prof_forename.}
      OPTIONAL { ?professor cpm:firstName ?prof_firstName.}
      BIND (IF (BOUND(?prof_forename), ?prof_forename, ?prof_firstName)  as ?forename)
      BIND (CONCAT(?forename, ' ',  ?surname) as ?label)

      OPTIONAL {?professor cpm:note ?note.}
      OPTIONAL {?professor cpm:furtherInformation ?furtherInformation.}
      OPTIONAL {?professor cpm:webLink ?webLink.}
      OPTIONAL {?professor rdfs:seeAlso ?seeAlso.}
      OPTIONAL {?professor cpm:references ?references.}
      OPTIONAL {?professor cpm:literature ?literature.}
      OPTIONAL {?professor cpm:alternativeWritingOfSurname ?alternativeWritingOfSurname.}
      OPTIONAL {?professor cpm:gnd ?gnd.}
  }
}
"""
print(query_sparql(data,endpoint))


# Basic Import of Lecturers including relevant attributes and pictures
#-----
33348-36002
--- 1.1767358779907227 seconds ---


In [3]:
data = """
# Basic Import of Lecturers Geburt und Tod
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>

CONSTRUCT {

# Lecturer
?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

# Stage of Life dates
?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.
?pcp_period rdfs:comment ?note.

# Stage of Life location
?pcp_period pcp:stageOfLifeLocation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.
?pcp_place pcp:derivedFrom ?place.

}

WHERE {
  
  GRAPH data: {  
    ?lecturer rdf:type pcp:Lecturer.
    FILTER contains(str(?lecturer),"/bamberg/")
    ?lecturer pcp:derivedFrom ?professor.
  }
  GRAPH bamberg: {  

      # Import all Professors relevant for Early Modern History
      ?professor cpm:hasPeriod ?period.
      ?period rdf:type ?period_type.
      FILTER(contains(str(?period_type),"Birth")||contains(str(?period_type),"Death"))

      BIND (IRI(REPLACE(REPLACE(str(?period),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?pcp_period). 
      BIND (IRI(REPLACE(str(?period_type),str(cpm:),str(pcp:))) AS ?pcp_period_type).


      #retreive basic properties from period of Life
      
      OPTIONAL {?period cpm:date ?date.}
      OPTIONAL {?period cpm:from ?from.}
      OPTIONAL {?period cpm:to ?to.}
      OPTIONAL {?period cpm:note ?note.}
      
    BIND( IF (contains(str(?period_type),'Birth'), CONCAT("geboren am ",str(?date)), CONCAT("gestorben am ",str(?date))) AS ?period_label)
      
      
      OPTIONAL {?period cpm:periodPlace ?place.
        ?place <http://schema.org/label> ?place_label.
        BIND (IRI(REPLACE(REPLACE(str(?place),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'place/'))) AS ?pcp_place). 
      }

    }
  
}

"""
print(query_sparql(data,endpoint))


# Basic Import of Lecturers Geburt und Tod
#-----
36002-39889
--- 2.0332157611846924 seconds ---


In [4]:
data = """
# Stage of Life: Ämter inkl. akademische Ämter und deren Orte
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>


CONSTRUCT   {

?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.


}
WHERE {

  GRAPH data: {  
    ?lecturer rdf:type pcp:Lecturer.
    FILTER contains(str(?lecturer),"/bamberg/")
    ?lecturer pcp:derivedFrom ?professor.
  }

  GRAPH bamberg: {  
  
      # Import all Professors relevant for Early Modern History
      ?professor cpm:hasPeriod ?period.
      ?period rdf:type cpm:PeriodOfLife.

      BIND (IRI(REPLACE(REPLACE(str(?period),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?pcp_period). 


      #retreive basic properties from period of Life
      
    
      OPTIONAL {?period cpm:periodNote ?period_label.
      
      BIND(
            IF( contains(?period_label,"Prof")||
            contains(?period_label,"dekan")||
            contains(?period_label,"Rektor")||
            contains(?period_label,"Präsident")||
            contains(?period_label,"rektor")||
            contains(?period_label,"Institut")||
            contains(?period_label,"institut")||
            contains(?period_label,"Rat")||
            contains(?period_label,"rat")||
            contains(?period_label,"Kommission")||
            contains(?period_label,"kommission")||
            contains(?period_label,"Senat")||
            contains(?period_label,"senat")||
            contains(?period_label,"Leiter")||
            contains(?period_label,"leiter"), pcp:AcademicOffice,
          
            IF( contains(?period_label,"Professor")||contains(?period_label,"Lehrer"), pcp:OfficeLecturer, pcp:Office))
          
	        AS ?pcp_period_type).  
        
      }
        
      OPTIONAL {?period cpm:date ?date.}
      OPTIONAL {?period cpm:from ?from.}
      OPTIONAL {?period cpm:to ?to.}


    }
}
"""
print(query_sparql(data,endpoint))


# Stage of Life: Ämter inkl. akademische Ämter und deren Orte
#-----
39889-42722
--- 1.4452893733978271 seconds ---


In [5]:
data = """
# Stage of Life: Karriereschritte inkl. Lehrstühle und deren Orte
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>


CONSTRUCT   {

# Lecturer
?pcp_period rdf:type ?pcp_period_type.
?pcp_period pcp:derivedFrom ?period.
?lecturer pcp:hasStageOfLife ?pcp_period.

# Stage of Life dates
?pcp_period pcp:date ?date.
?pcp_period pcp:from ?from.
?pcp_period pcp:to ?to.
?pcp_period rdfs:label ?period_label.


}
WHERE {

  GRAPH data: {  
    ?lecturer rdf:type pcp:Lecturer.
    FILTER contains(str(?lecturer),"/bamberg/")
    ?lecturer pcp:derivedFrom ?professor.
  }

  
  GRAPH bamberg: {  
  

    ?professor cpm:hasPeriod ?period.
    ?period rdf:type cpm:Career.

    BIND (IRI(REPLACE(REPLACE(str(?period),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?pcp_period). 

    OPTIONAL {
      ?period cpm:periodNote ?period_label.
      BIND( IF(contains(?period_label,"Prof"), pcp:ChairProfessorship, pcp:OfficeLecturer) AS ?pcp_period_type).  
    }
        
    OPTIONAL {?period cpm:date ?date.}
    OPTIONAL {?period cpm:from ?from.}
    OPTIONAL {?period cpm:to ?to.}


  }
}

"""
print(query_sparql(data,endpoint))


# Stage of Life: Karriereschritte inkl. Lehrstühle und deren Orte
#-----
42722-47122
--- 2.1289074420928955 seconds ---


In [6]:
data = """
# Vervollständigen der Körperschaften zu Periods of Life
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


CONSTRUCT   {

# Beziehung zu Stage of Life

?pcp_stage ?pcp_bodyprop ?pcp_body.

# Körperschaft: Fakultät oder Hochschule
?pcp_body rdfs:label ?body_label.
?pcp_body owl:sameAs ?body_sameas.
?pcp_body rdfs:seeAlso ?body_link.
?pcp_body rdfs:seeAlso ?body_homepage.
?pcp_body rdf:type ?pcp_bodytype.


# Location
?pcp_body pcp:locationCorporation ?pcp_place.
?pcp_place rdf:type pcp:Location.
?pcp_place rdfs:label ?place_label.


}
WHERE {
  GRAPH data: {
    ?person pcp:hasStageOfLife ?pcp_stage.
    FILTER contains(str(?pcp_stage),'/bamberg/')
    ?pcp_stage pcp:derivedFrom ?period .
    GRAPH bamberg: {  
    
        ?period cpm:periodBody ?body.
        ?body cpm:typeOfBody ?body_type.
        FILTER ((contains(?body_type,"Fakultät")) || (contains(?body_type,"Hochschule")))
        BIND ( IF( contains(?body_type,"Fakultät"), pcp:Faculty, pcp:University) AS ?pcp_bodytype )
        BIND ( IF( contains(?body_type,"Fakultät"), pcp:hasFaculty, pcp:hasUniversity) AS ?pcp_bodyprop )
        BIND (IRI(REPLACE(REPLACE(str(?body),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?pcp_body). 
        
        OPTIONAL { ?body <http://schema.org/label> ?body_label. }
        OPTIONAL { ?body foaf:homepage ?body_homepage.}        
        OPTIONAL { ?body owl:sameAs ?body_sameas.}
        OPTIONAL { ?body cpm:weblink ?body_link.}


        OPTIONAL {
            ?body cpm:bodyPlace ?place.
            ?place <http://schema.org/label> ?place_label.
            BIND (IRI(REPLACE(REPLACE(str(?place),'#','_'),'http://professorenkatalog.uni-bamberg.de/rdf/',CONCAT(str(data:),'bamberg/'))) AS ?pcp_place). 
        }
    }
  }
}


"""
print(query_sparql(data,endpoint))


# Vervollständigen der Körperschaften zu Periods of Life
#-----
47122-47988
--- 1.10023832321167 seconds ---


In [7]:
data = """
# Vervollständigung Körperschaften
#-----

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pcp: <http://pcp-on-web.de/ontology#>
PREFIX data: <http://data.pcp-on-web.de/v.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX cpm: <http://catalogus-professorum.org/cpm/2/>
PREFIX bamberg: <http://professorenkatalog.uni-bamberg.de/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

CONSTRUCT   {

# Fakultät
?pcp_faculty rdfs:label ?faculty_label.
?pcp_faculty owl:sameAs ?faculty_sameas.
?pcp_faculty rdfs:seeAlso ?faculty_link.

}
WHERE {
  GRAPH data: {
    ?pcp_faculty rdf:type pcp:Faculty.
    FILTER contains(str(?pcp_faculty),'/bamberg/')
    ?pcp_faculty pcp:derivedFrom ?body
    GRAPH bamberg: {  
        OPTIONAL {?body <http://schema.org/label> ?faculty_label.}
        OPTIONAL {?body owl:sameAs ?faculty_sameas.}
        OPTIONAL {?body cpm:weblink ?faculty_link.}
    }
  }
}

"""
print(query_sparql(data,endpoint))

        


# Vervollständigung Körperschaften
#-----
47988-47988
--- 0.3029043674468994 seconds ---


In [8]:
# Korrekturen